Testing SuperK SELECT control via Python. We are using NKT Photonics SuperK Compact laser with SELECT tunable filter. Laser is on COM3.

On Rayleigh, SDK documents can be found under Users/Public/Public Documents/NKT Photonics

SuperK Compact info is on page 59 of the user manual.

Python interface doesn't work while Control software is connected.

In [5]:
from controls import *

Loading x64 DLL from: C:\Users\Public\Documents\NKT Photonics\SDK\NKTPDLL\x64\NKTPDLL.dll


In [6]:
scan_ports()

Following ports has modules: COM3
Comport: COM3 Device type: 0x0074 at address: 1
Comport: COM3 Device type: 0x0067 at address: 16
Comport: COM3 Device type: 0x0066 at address: 17
Close result:  0:OPSuccess


In [7]:
get_interlock()

Waiting for interlock to reset.


In [8]:
reset_interlock()

Interlock is OK.


In [13]:
trig_mode()

Laser mode: Internal frequency generator


In [10]:
emission_on()

Setting emission ON. 0:RegResultSuccess


In [12]:
overall_power(20)

Setting overall power level to 20%.


In [14]:
emission_off()

Setting emission OFF. 0:RegResultSuccess


In [ ]:
# Function to check the status based on the mapping
def check_device_status(value):
    # Use the get method to provide a default message for unknown values
    status_mapping = {
    0: "Device is off.",
    1: "Device is on.",
    2: "Device is broken."
    }

    message = status_mapping.get(value, "Unknown status.")
    print(message)

# Example usage
status_value = int(input("Enter the device status (0: off, 1: on, 2: broken): "))
check_device_status(status_value)

"All wavelength and amplitude settings are sent to the RF Driver, not to the SuperK SELECT"
First channel is controlled with 0x90, second with 0x91, and so on up to 0x97 for the eighth channel. 
Only use the element on index 0. Can send only one 32 bit integer to change this value. 

In [100]:
COMport = 'COM3' # depends on the port the device is connected to
COMPACT_devID = 1 # fixed for the SuperK COMPACT
SELECT_devID = 16 # fixed for the SuperK SELECT

def crystal_temp():
    """Read the current crystal temperature in degrees Celsius."""
    result = registerReadS16(COMport, SELECT_devID, 0x38, -1)
    temperature = result[1] # temperature given in 0.1 degrees C
    print('Crystal temperature: ', temperature/10, 'degrees C.', RegisterResultTypes(result))

crystal_temp()

Crystal temperature:  0.0 degrees C. Unknown result


In [ ]:
def set_channel(channel, wavelength, amplitude):          # NOT WORKING
    """Specify the channel (1-8), then set the wavelength in nm and amplitude as a percent for that channel."""
    wavelength_channel_ID = int(channel-1)
    wavelength_address = f"0x9{wavelength_channel_ID}"
    wavelength_address = int(wavelength_address, 16)
    wavelength_pm = int(wavelength*1000) # convert nm to pm as the input format

    amplitude_channel_ID = int(channel-1)
    amplitude_address = f"0xB{amplitude_channel_ID}"
    amplitude_address = int(amplitude_address, 16)
    amplitude_percent = int(amplitude*10) # convert to tenths of a percent as the imput format

    result1 = registerWriteReadU32(COMport, SELECT_devID, wavelength_address, wavelength_pm, 0)
    result2 = registerWriteReadU16(COMport, SELECT_devID, amplitude_address, amplitude_percent, -1)

    print(f'Channel {channel} wavelength set to:', wavelength, 'nm, amplitude set to:', amplitude, '%.', RegisterResultTypes(result1), RegisterResultTypes(result2))

In [84]:
set_channel(1, 1500, 10)

Channel 1 wavelength set to: 1500 nm, amplitude set to: 10 % Unknown result Unknown result


In [ ]:
def read_connected_crystal():  # CHECK IF WORKING
    """Read the connected crystal."""
    result = registerReadS8(COMport, SELECT_devID, 0x75, -1)
    crystal_type = result[1]

    if crystal_type==0:
        print('No crystal is connected to the RF driver.')
    else:
        print('Connected to crystal:', crystal_type, RegisterResultTypes(result))

read_connected_crystal()

No crystal is connected to the RF driver.


In [66]:
read_connected_crystal()

(4, 0)
Connected crystal type: 0 Unknown result


TODO: double check the device address for RF driver

In [3]:
def RF_power_on():
    """Turn on power output to the RF driver to SuperK SELECT."""
    result = registerWriteReadU8(COMport, SELECT_devID, 0x30, 1, -1)
    print('RF power output is on.', RegisterResultTypes(result))

In [ ]:
def RF_power_off():
    """Turn off power output to the RF driver to SuperK SELECT."""
    result = registerWriteReadU8(COMport, SELECT_devID, 0x30, 0, -1)
    print('RF power output is off.', RegisterResultTypes(result))

In [80]:
# TRY GETTING A RESPONSE FROM THE DIFFERENT RESULT TYPES

def wavelength_min1():
    """Crystal 1 (VIS/NIR) minimum wavelength in nm."""
    result = registerReadU32(COMport, SELECT_devID, 0x90, -1)
    wavelength = result[1]/1000 # convert pm to nm
    print(f'Minimum wavelength for crystal 1: {int(wavelength)} nm.', RegisterStatusTypes(result))

wavelength_min1()

Minimum wavelength for crystal 1: 500 nm. Unknown status
